# Colab Code 

In [1]:
# # Install the PyDrive wrapper & import libraries.
# # This only needs to be done once per notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download a file based on its file ID.
# #https://drive.google.com/open?id=1_Xo1PTfFgY3KgPVxo5gzghj_pRzp5gCj
# # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# file_id = '1_Xo1PTfFgY3KgPVxo5gzghj_pRzp5gCj'
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile("images_Data.zip")
# # print('Downloaded content "{}"'.format(downloaded.GetContentString()))
# !unzip images_Data.zip


In [2]:
# # https://drive.google.com/open?id=1Q5j3Hy3P-0K5Nip0ztNmsWpKu8oToYEy
# file_id = '1Q5j3Hy3P-0K5Nip0ztNmsWpKu8oToYEy'
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile("testing_data.pickle")

In [3]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [4]:
# import os
# print(len(os.listdir("./images")))
# !cp -a ./aug_images/. ./images/
# print(len(os.listdir("./images")))

# importing dependencies 

In [6]:
import keras as ks
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle as pkl
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier

Using TensorFlow backend.


In [0]:
from model import create_model
from preprocessing import preprocess

# Data handling 

In [8]:
orig_labels=[]
with open("./labels.pickle","rb") as file:
    orig_labels=pkl.load(file)
    
aug_labels=[]
with open("./augments_label.pickle","rb") as file:
    aug_labels=pkl.load(file)
    
labels=orig_labels+aug_labels

with open("./testing_data.pickle","rb") as file:
    test_data,test_labels=pkl.load(file)
    
len(labels)

1065

In [9]:
def simplify_labels(labels):
    new_labels=[]
    for image in labels:
        for num in range(image[1][0]):
            new_labels.append((image[0],(1,[image[1][1][num]],[image[1][2][num]])))
    return new_labels

labels=simplify_labels(labels)
# np.random.shuffle(labels)
len(labels)

1920

# Approach 1 - VGG extractor + Classic algo

In [0]:
base_model=ks.applications.VGG16(include_top=False,input_shape=(224,224,3))

In [0]:
gen=preprocess(labels,"./images/",batch_size=total_data)
data,t=next(gen)

In [6]:
data_feat=base_model.predict(data)
data_feat=data_feat.reshape((1920,-1))

In [0]:
cls1=LinearSVC()
cls2=KNeighborsClassifier(n_neighbors = 5)

In [41]:
cls1.fit(data_feat,t)
cls2.fit(data_feat,t)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
test_feat=base_model.predict(test_data)
test_feat=test_feat.reshape((139,-1))
test_feat.shape

In [42]:
cls1.score(test_feat,test_labels),cls2.score(test_feat,test_labels)

(0.6258992805755396, 0.6330935251798561)

In [43]:
print(classification_report(test_labels,cls1.predict(test_feat)))
print(classification_report(test_labels,cls2.predict(test_feat)))

              precision    recall  f1-score   support

           0       0.52      0.80      0.63        55
           1       0.80      0.51      0.62        84

    accuracy                           0.63       139
   macro avg       0.66      0.66      0.63       139
weighted avg       0.69      0.63      0.63       139

              precision    recall  f1-score   support

           0       0.53      0.60      0.56        55
           1       0.71      0.65      0.68        84

    accuracy                           0.63       139
   macro avg       0.62      0.63      0.62       139
weighted avg       0.64      0.63      0.64       139



# Approach 2 - Deep learning

In [0]:
# model=create_model((224,224,3))

# batch_size=128
# epochs=7
# steps_per_epoch=int(total_data/(batch_size))
# steps_per_epoch

# model.fit_generator(preprocess(labels,"./images/",batch_size=batch_size),steps_per_epoch=steps_per_epoch,epochs=epochs-5)

# preds=model.predict_classes(test_data)
# print(classification_report(test_labels,preds))
# (test_labels==preds.reshape((-1))).sum()/preds.shape[0]

# model.save("./model_1.h5")


In [0]:

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Create & upload a text file.
# uploaded = drive.CreateFile({'title': 'IP_model_2.h5'})
# uploaded.SetContentFile("model_1.h5")
# uploaded.Upload()
# # print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
# with open("./labels_total.pickle","wb") as file:
#     pkl.dump(labels,file)

# img=cv2.resize(cv2.imread("./aug_images/IMG-20190708-WA0008_4.jpg"),(224,224))
# plt.imshow(img),model.predict([[img]])